In [1]:
from Twitter_keeper import PullTweetsData

puller = PullTweetsData()
puller.connectToDB("twitter_keeper", "tweets")

puller.find_multi("", "", "", "Bangkok", "", "2023.2.12.17.0.0", "2023.2.12.17.40.0")

[{'_id': ObjectId('63e8c268904b045f69e18d56'),
  'tweet_author': 'Tinkindeed',
  'tweet_create_at': datetime.datetime(2023, 2, 12, 10, 12, 51),
  'hashtag': '',
  'keyword': '#ใส่นัวแฟมิลี่',
  'text': 'เอาไปวิจัยได้ไหมว่าทำไมคนไทยถึงคาดหวังการขับเคลื่อนสังคมจากพี่เอกนัก\u200b ทั้งๆที่แกก็ชัดเจนนะว่าแกมาเล่นเกม\u200b ในทางกลับกันคนที่มีชื่อเสียง\u200b ดังกว่าแกพับลิคกว่า\u200bแกมากมายที่ติ่งแฮรี่\u200bติ่งแบบตะโกนกลับไม่เคยโดนทัวลงแบบที่แกโดน\u200bหรือว่าแกเลือดกรุปบี\u200bเหรอ\u200b งง\n#HRK #ใส่นัวแฟมิลี่',
  'tweet_location': 'Bangkok Thailand'},
 {'_id': ObjectId('63e8ebcf904b045f69e1c195'),
  'tweet_author': 'leadtutorth',
  'tweet_create_at': datetime.datetime(2023, 2, 12, 10, 33),
  'hashtag': '#ติวสอบ#ไอเอล#IELTS#หาติวเตอร์#dek66#dek65#TCAS66#TCAS65#โทอิค#แกท#TGAT#TPAT#ติวเตอร์#สอนพิเศษ',
  'keyword': '#dek66',
  'text': 'รับสอนพิเศษทุกวิชา\nตั้งแต่ประถม-วัยทำงาน\nมีชีทให้ทุกวิชา ติวเตอร์จากมหาลัยชั้นนำ\nเลือกติวเตอร์ได้นะคะ\nเรียนเดี่ยว คู่ กลุ่ม \nวางแผนเป็นรายบุคคล ปรึกษาฟรี